In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas


spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

#filtered_df = df.filter(df.installation_id is unique)
# display(df.select("installation_id").distinct())
# display(df.select("tag").distinct())

df = spark.sql("""
               select distinct installation_id
               from test_unify_analytics.bronze.logs_partition_1
              """)
list = df.toPandas()['installation_id'].tolist()
print(list)

# df = spark.table('test_unify_analytics.bronze.logs_partition_1') \
#         .select('installation_id', 'tag', 'unix_timestamp', 'values') \
#         .filter((col("tag") == 242) & (col('values')[5] == 14))
# df = spark.sql("""
#                 select installation_id, tag, unix_timestamp, values
#                 from test_unify_analytics.bronze.logs_partition_1
#                 where tag = 242 AND values[5] = 14
#                 """)
# display(df)




In [0]:
%sql
select installation_id, tag, count(tag)
from test_unify_analytics.bronze.logs_1
where installation_id = 'a037a00000pwUFKAA2' and (tag = 248 or tag = 249)
group by installation_id, tag
order by tag asc;

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

ps_df = spark.table('test_unify_analytics.bronze.logs_partition_1')
ps_df = ps_df.filter((ps_df.installation_id == 'a030Y00000KPfzuQAD') & (ps_df.tag == 242))

# ps_df1 = ps_df.select(ps_df.installation_id, ps_df.tag, ps_df.unix_timestamp, ps_df.values[1].alias('bin num'))
# pd_df1 = ps_df1.toPandas()
# pd_df1 = pd_df1.sort_values(by=['bin num', 'unix_timestamp'])
# display(pd_df1)
# print('num rows: ', pd_df1.size)

ps_df2 = ps_df.select(ps_df.values[0].alias('bin num'), ps_df.values[5].alias('mode')).filter(ps_df.values[5] == 14)
ps_df2 = ps_df2.groupBy(['bin num', 'mode']).count()

pd_df2 = ps_df2.toPandas().sort_values(by=['count'], ascending=False)
display(pd_df2)

In [0]:
import pandas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Installation Data").getOrCreate()
        
# for i in range (0,4):
# values[5] == 14 --> bin in PORT
df = spark.table('test_unify_analytics.bronze.logs_partition_1') \
       .select('installation_id', 'tag', 'unix_timestamp', 'values') \
       .filter((col('installation_id') == 'a031v00001GpVAEAA3') & 
               (col('tag') == 242) & 
               (col('values')[5] == 14))


df1 = df.withColumn('bin num', col('values')[0]) \
       .withColumn('unix timestamp', col('unix_timestamp'))
window_spec = Window.partitionBy('bin num').orderBy('unix_timestamp')
df1 = df1.withColumn('prev_timestamp', lag('unix_timestamp').over(window_spec)) \
       .withColumn('time diff (sec)', col('unix_timestamp') - col('prev_timestamp'))

result_df = df1.groupBy('bin num').agg(round(mean('time diff (sec)')).alias('avg time between presentations (sec)'))
result_df = result_df.withColumn('avg time between presentations (min)', round(col('avg time between presentations (sec)') / 60))
pd = result_df.toPandas().sort_values(by=['avg time between presentations (sec)'])
display(pd)

# pd.to_csv('/dbfs/FileStore/tables/a031v00001GpVAEAA3.csv')
# # retrieve_filepath = '/files/tables/a031v00001GpVAEAA3.csv'
# url = 'https://adb-4786482868192795.15.azuredatabricks.net/files/tables/a031v00001GpVAEAA3.csv'
# response = requests.get(url)
# print(response)

In [0]:
import pandas as pd

# dbutils.fs.rm('/FileStore/tables/', recurse=True)
# dbutils.fs.mkdirs('dbfs:/FileStore/tables')
files = dbutils.fs.ls('/FileStore/tables/')
display(files)
# l = [f.name for f in files]
# print('num files: ', len(l))
# display(pd.DataFrame(l, columns=['installations']))